<a href="https://colab.research.google.com/github/riddhi-mehta-1721/NLP_Project/blob/master/Book_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP PROJECT

## RIDDHI MEHTA - J030
## HUSAIN GHADIALI - J056

# Importing Libraries

In [12]:
import pandas as pd
import numpy as np

import re
import pickle
import json
import io

from io import StringIO

import seaborn as sns
import matplotlib.pyplot as plt

import pydotplus
import pydot

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


from tensorflow.python.keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout, Conv1D, MaxPooling1D, GRU
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import tokenizer_from_json
from keras.preprocessing.sequence import pad_sequences
# from keras.models import Model
# from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout
# from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils import plot_model  
from keras.utils.vis_utils import model_to_dot

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

import warnings
warnings.filterwarnings("ignore")

import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Mounting Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Predicting on Kaggle Test Dataset

In [0]:
def preprocess(sentence):

  text = sentence
  text = text.lower()
  text = re.sub(r"what's", "what is ", text)
  text = re.sub(r"\'s", " ", text)
  text = re.sub(r"\'ve", " have ", text)
  text = re.sub(r"can't", "cannot ", text)
  text = re.sub(r"n't", " not ", text)
  text = re.sub(r"i'm", "i am ", text)
  text = re.sub(r"\'re", " are ", text)
  text = re.sub(r"\'d", " would ", text)
  text = re.sub(r"\'ll", " will ", text)
  text = re.sub(r"\'scuse", " excuse ", text)
  text = re.sub('\W', ' ', text) # substitutes non word character with a space
  text = re.sub('\s+', ' ', text)
  text = text.strip(' ')
  text = re.sub(r"[0-9]"," ",text) # substitutes numbers with a blank space
  
  my_tokens = list() # Initialising a list of tokens that will be formed
  stop_words= set(stopwords.words('english')) # Extracting english stopwords from nltk corpus
  
  tokens = word_tokenize(text, preserve_line=False)

  for token in tokens:
    if (token not in stop_words):
      my_tokens.append(token)

  final = ' '.join([str(elem) for elem in my_tokens]) # makes it an entire string

  return final

In [0]:
k_test = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/kaggle_test.csv', 
                  dtype = {'id' : str, 'comment_text': str}   )

In [14]:
k_test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [0]:
x_test = pd.DataFrame(k_test["comment_text"])

x_test["comment_text"] = x_test["comment_text"].apply(preprocess)

In [0]:
train = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/train.csv")
train.drop(columns = "Unnamed: 0", inplace = True)
X_train = train["comment_text"]

max_features = 20000
tokenizer = Tokenizer(num_words = max_features, char_level = True) 
tokenizer.fit_on_texts(list(X_train))

In [0]:
test = x_test["comment_text"]

test = tokenizer.texts_to_sequences(test)  

In [0]:
maxlen = 100
test = pad_sequences(test, maxlen = maxlen, padding = "post")

In [19]:
model = load_model(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Models/nn4.hdf5")
pred = model.predict(test, verbose = 1)

4787/4787 [==============================] - 140s 29ms/step


In [20]:
pred

array([[9.98230636e-01, 3.78732234e-01, 9.95744884e-01, 3.83478068e-02,
        9.53207374e-01, 1.62006870e-01],
       [8.34740877e-01, 2.08991636e-02, 1.68355972e-01, 1.38262389e-02,
        2.54785955e-01, 5.29731847e-02],
       [8.77261698e-01, 2.34035738e-02, 3.05176526e-01, 1.62008721e-02,
        3.22717875e-01, 4.08973806e-02],
       ...,
       [3.47051546e-02, 6.06870803e-04, 7.28017464e-03, 1.09573384e-03,
        1.23752151e-02, 2.59249192e-03],
       [1.15491375e-02, 3.89082212e-04, 3.01267439e-03, 5.43617352e-04,
        4.73518018e-03, 1.11420732e-03],
       [7.42655277e-01, 1.16674146e-02, 4.91627991e-01, 8.05689022e-03,
        3.80228251e-01, 1.68845709e-02]], dtype=float32)

In [21]:
df = pd.DataFrame.from_records(pred)
df.columns = ["toxic","severe_toxic", "obscene", "threat", "insult", "identity_hate"]
df.shape

(153164, 6)

In [22]:
df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0.998231,0.378732,0.995745,0.038348,0.953207,0.162007
1,0.834741,0.020899,0.168356,0.013826,0.254786,0.052973
2,0.877262,0.023404,0.305177,0.016201,0.322718,0.040897
3,0.007811,0.000280,0.001792,0.000366,0.002636,0.000558
4,0.755440,0.007590,0.099156,0.007321,0.156043,0.028790


In [0]:
outer_list = []
for y in pred:
  inner_list = []
  for i in y:
    if i >= 0.5:
      inner_list.append(1)
    else:
      inner_list.append(0)
  outer_list.append(inner_list)

df = pd.DataFrame.from_records(outer_list)
df.columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

In [82]:
df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,0,1,0,1,0
1,1,0,0,0,0,0
2,1,0,0,0,0,0
3,0,0,0,0,0,0
4,1,0,0,0,0,0


In [0]:
df = pd.concat([x_test["comment_text"], df], axis = 1)

In [24]:
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,yo bitch ja rule succesful ever whats hating s...,0.998231,0.378732,0.995745,0.038348,0.953207,0.162007
1,rfc title fine imo,0.834741,0.020899,0.168356,0.013826,0.254786,0.052973
2,sources zawe ashton lapland,0.877262,0.023404,0.305177,0.016201,0.322718,0.040897
3,look back source information updated correct f...,0.007811,0.000280,0.001792,0.000366,0.002636,0.000558
4,anonymously edit articles,0.755440,0.007590,0.099156,0.007321,0.156043,0.028790


In [88]:
df[(df.toxic == 0) & (df.severe_toxic == 0)]

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
3,look back source information updated correct f...,0,0,0,0,0,0
5,thank understanding think highly would revert ...,0,0,0,0,0,0
6,please add nonsense wikipedia edits considered...,0,0,0,0,0,0
8,fool believe numbers correct number lies ponde...,0,0,0,0,0,0
9,double redirects fixing double redirects blank...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
153157,disagree soviet railways need article administ...,0,0,0,0,0,0
153160,throw field home plate get faster throwing cut...,0,0,0,0,0,0
153161,okinotorishima categories see changes agree co...,0,0,0,0,0,0
153162,one founding nations eu germany law return qui...,0,0,0,0,0,0


# Making data ready for submission

In [0]:
sample_submission = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/sample_submission.csv", 
                                dtype = {'id': str, 'comment_text':str})

In [26]:
sample_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.5,0.5,0.5,0.5,0.5,0.5
1,0000247867823ef7,0.5,0.5,0.5,0.5,0.5,0.5
2,00013b17ad220c46,0.5,0.5,0.5,0.5,0.5,0.5
3,00017563c3f7919a,0.5,0.5,0.5,0.5,0.5,0.5
4,00017695ad8997eb,0.5,0.5,0.5,0.5,0.5,0.5


In [0]:
final_submission = pd.concat([sample_submission["id"],df.iloc[:,1:]], axis = 1)

In [28]:
final_submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998231,0.378732,0.995745,0.038348,0.953207,0.162007
1,0000247867823ef7,0.834741,0.020899,0.168356,0.013826,0.254786,0.052973
2,00013b17ad220c46,0.877262,0.023404,0.305177,0.016201,0.322718,0.040897
3,00017563c3f7919a,0.007811,0.000280,0.001792,0.000366,0.002636,0.000558
4,00017695ad8997eb,0.755440,0.007590,0.099156,0.007321,0.156043,0.028790


In [0]:
final_submission.to_csv(r"/content/drive/My Drive/Colab Notebooks/NLP - Sem 6/Project/Datasets/submission.csv", index = False)

# ----------------- THE END -----------------